# Εργαστηριακή Άσκηση 2
Σκοπός της δεύτερης σειράς ασκήσεων είναι η εξοικείωση με τις συναρτήσεις σχεδιασμού φίλτρων πεπερασμένης κρουστικής απόκρισης (FIR) και την υλοποίησή τους στο MATLAB. **Προτού ξεκινήσετε την άσκηση θα πρέπει να μελετήσετε με προσοχή το Κεφάλαιο 1 και, ειδικότερα, την παράγραφο 1.3 του τεύχους του μαθήματος.** Το MATLAB (www.mathworks.com) είναι ένα διαδραστικό εμπορικό πρόγραμμα (Windows, Linux, Unix) με το οποίο μπορείτε να κάνετε εύκολα αριθμητικές πράξεις με πίνακες. Μπορεί να το έχετε εγκατεστημένο τοπικά, στον προσωπικό σας υπολογιστή ή να εργάζεστε σε κάποιο Εργαστήριο Προσωπικών Υπολογιστών (ΕΠΥ) της Σχολής σας που διαθέτει το συγκεκριμένο λογισμικό.


## Μέρος 1: Εισαγωγή
Στο MATLAB οι συναρτήσεις $fft$ και $ifft$ υποθέτουν ζεύγος μετασχηματισμού Fourier $x(t)$ και $X(f)$ υπολογισμένων σε μη αρνητικά διαστήματα 
$ t=[0:N-1]ts $ και $ f=[0:N-1]fo. $ Όπως έχετε ήδη δει στην Εργαστηριακή άσκηση 1, το άνω μισό μέρος του διαστήματος συχνοτήτων αντιστοιχεί στις αρνητικές συχνότητες του σήματος, όταν υπολογίζουμε το $X(f)$ με τη βοήθεια της συνάρτησης $fft$ (για σήματα πραγματικών τιμών, αυτά τα δυο μισά είναι κατοπτρικά ως προς το μέσο του διαστήματος). Ακριβώς το ίδιο ισχύει και για το άνω μισό μέρος του χρονικού διαστήματος, όταν το σήμα $x(t)$ προκύπτει από τον αντίστροφο μετασχηματισμό Fourier μέσω της $ifft$.

Το MATLAB διαθέτει τη συνάρτηση $fftshift$ για να ολισθήσει κυκλικά τις τιμές του σήματος ή του μετασχηματισμού Fourier, ώστε να αντιστοιχούν σε κεντραρισμένα στο μηδέν αμφίπλευρα διαστήματα, δηλαδή, στις χρονικές στιγμές $tb=[-ceil((N-1)/2): floor((N-1)/2)]ts$ ή στις συχνότητες $fb=[–ceil((N-1)/2): floor((N-1)/2)]fo$. Με τον τρόπο αυτό μπορούμε να παράγουμε τα $xb(t)$ και $Xb(f)$ που αντιστοιχούν στην αμφίπλευρη αναπαράσταση του σήματος και του μετασχηματισμού Fourier.

Για να κατανοήσετε τα ανωτέρω θεωρείστε το διάνυσμα $[1 2 3 4]$ ως το αποτέλεσμα του $FFT$ μήκους 4. Τότε, το πρώτο στοιχείο (1) είναι ο όρος dc, το τρίτο στοιχείο (3) είναι το σημείο στο μισό της συχνότητας δειγματοληψίας $fs/2$, που μπορεί να εκληφθεί ότι αντιστοιχεί είτε στην $–fs/2$ είτε στην $+fs/2$. Τα στοιχεία 2 και 4 αντιστοιχούν στις συχνότητες $+fs/4$ και $–fs/4$. Εφαρμόζοντας την $fftshift$, το στοιχείο 3 εμφανίζεται πρώτο, που σημαίνει ότι στο MATLAB αντιστοιχεί στην αρνητική συχνότητα $–fs/2$, το επόμενο στοιχείο 4 αντιστοιχεί στη συχνότητα $–fs/4$ ακολουθούμενο από το dc και τη συχνότητα $+fs/4$. Για ένα μετασχηματισμό περιττού μήκους, δεν υφίσταται σημείο για το $±fs/2$. Έτσι για το διάνυσμα $[1 2 3]$, η εφαρμογή της $fftshift$ θα δώσει τα στοιχεία που αντιστοιχούν στις συχνότητες $–fs/3, 0, +fs/3$.

Εκτός του ότι παράγουν εξόδους με τις αρνητικές συχνότητες ή χρόνους στο άνω μισό του διανύσματος, αμφότερες οι συναρτήσεις $fft$ και $ifft$ αναμένουν ως είσοδο διάνυσμα με την ίδια μορφή, αφού προφανώς ισχύουν οι ταυτότητες

<center>

$h == \text{ifft}(\text{fft}(h))$ και $H == \text{fft}(\text{ifft}(H))$

</center>


Η πρώτη υποδεικνύει ότι η είσοδος της $ifft$ πρέπει να είναι αντεστραμμένη, όπως την παράγει η $fft$, και η δεύτερη ότι η είσοδος της $fft$ πρέπει να είναι αντεστραμμένη, όπως την παράγει η $ifft$.

Στο επόμενο σχήμα βλέπετε παραστατικά ένα ημιτονικό σήμα που έχει πολλαπλασιασθεί με παράθυρο Blackman τόσο στην αμφίπλευρη, όσο και την μονόπλευρη αναπαράστασή του.

In [ ]:
import plotly.graph_objs as go
import numpy as np
from plotly.subplots import make_subplots


# Seed for reproducibility
np.random.seed(0)

# Generate a windowed random data
t = np.linspace(-40, 40, 80)  # 80 samples
window = np.zeros_like(t)
window[30:50] = 1  # non-zero between indices 30 and 50
random_data = np.random.randn(len(t))
x_t_windowed = window * random_data  # Apply window to random data

# Create a figure
fig = make_subplots()

# Add stem lines - one line plot for each stem
for i in range(len(t)):
    if window[i] != 0:  # Only plot stems for the windowed part
        fig.add_trace(go.Scatter(x=[t[i], t[i]], y=[0, x_t_windowed[i]], mode='lines', line=dict(color='blue'), showlegend=False))

# Add markers at the top of stems
fig.add_trace(go.Scatter(x=t, y=x_t_windowed, mode='markers', name='Windowed Random Data', marker=dict(color='blue')))

# Update yaxis and xaxis properties
fig.update_yaxes(title_text="Amplitude")
fig.update_xaxes(title_text="Time (samples)")

# Update layout
fig.update_layout(height=600, width=800, title_text="Windowed Random Data with Stem Lines")
fig.show(renderer='notebook')


<div style="text-align: center;">
    <img src="../content/images/lab2_1.png" alt="lab2_1.png">
</div>

In [ ]:
import plotly.graph_objs as go
import numpy as np
from plotly.subplots import make_subplots


# Seed for reproducibility
np.random.seed(0)

# Generate windowed random data
t = np.linspace(-40, 40, 80)  # 80 samples
window = np.zeros_like(t)
window[30:50] = 1  # non-zero between indices 30 and 50
random_data = np.random.randn(len(t))
x_t_windowed = window * random_data  # Apply window to random data

# Μετατροπή των αρνητικών χρόνων στο τέλος
negative_time_indices = t < 0
positive_time_indices = t >= 0

# Συνδυασμός των δεδομένων με τα αρνητικά στο τέλος
t_reordered = np.concatenate((t[positive_time_indices], t[negative_time_indices]))
x_t_windowed_reordered = np.concatenate((x_t_windowed[positive_time_indices], x_t_windowed[negative_time_indices]))

# Create a figure
fig = make_subplots()

# Add stem lines for reordered data
for i in range(len(t_reordered)):
    fig.add_trace(go.Scatter(x=[t_reordered[i], t_reordered[i]], y=[0, x_t_windowed_reordered[i]], mode='lines', line=dict(color='blue'), showlegend=False))

# Add markers at the top of stems for reordered data
fig.add_trace(go.Scatter(x=t_reordered, y=x_t_windowed_reordered, mode='markers', name='Reordered Windowed Random Data', marker=dict(color='red')))

# Update yaxis and xaxis properties
fig.update_yaxes(title_text="Amplitude")
fig.update_xaxes(title_text="Time (samples)")

# Update layout
fig.update_layout(height=600, width=800, title_text="Reordered Windowed Random Data with Stem Lines")
fig.show(renderer='notebook')


Οι αντίστοιχοι μετασχηματισμοί Fourier, σε αμφίπλευρη και μονόπλευρη αναπαράσταση, φαίνονται στο επόμενο σχήμα:

<div style="text-align: center;">
    <img src="../content/images/lab2_2.png" alt="lab2_2.png">
</div>

Όταν τα $x(t)$ και $X(f)$ παράγονται από το MATLAB, δεν χρειάζεται κάποια ιδιαίτερη προσοχή, πλην της κυκλικής ολίσθησης σε περίπτωση που θέλουμε π.χ. να σχεδιάσουμε το αμφίπλευρο φάσμα ή σήμα. Όταν όμως ένα εκ των $x(t)$ ή $X(f)$ ορίζεται από τον χρήστη απαιτείται περισσότερη προσοχή, διότι, συνήθως χρησιμοποιούνται τα αμφίπλευρα σήματα ή φάσματα. Μπορείτε να μεταβείτε από τη μία αναπαράσταση στην άλλη ως εξής:

$x = ifftshift(xb)$, $X = fft(x)$, $Xb = fftshift(X)$, εάν ξεκινάτε από αμφίπλευρο σήμα και θέλετε να καταλήξετε σε αμφίπλευρο φάσμα, και

$X = ifftshift(Xb)$, $x = ifft(X)$, $xb = fftshift(x)$, εάν ξεκινάτε από αμφίπλευρο φάσμα και θέλετε να καταλήξετε σε αμφίπλευρο σήμα,

όπου η συνάρτηση $ifftshift$ του MATLAB εκτελεί την αντίστροφη λειτουργία της $fftshift$. Όταν το $N$ είναι άρτιο, οι $fftshift$ και $ifftshift$ δίνουν το ίδιο αποτέλεσμα. Όταν όμως το $N$ είναι περιττό αυτό δεν ισχύει και χρειάζεται προσοχή στη χρήση τους. Στην πράξη, η προσεκτική εφαρμογή των ανωτέρω έχει σημασία όταν υπολογίζεται η φάση του φάσματος. Το πλάτος του φάσματος δεν επηρεάζεται από την κυκλική ολίσθηση των στοιχείων που προκαλούν οι $fftshift$ και $ifftshift$ (δείτε ιδιότητες DFT).

### Εξάσκηση
Δοκιμάστε στο παράθυρο εντολών τα ακόλουθα προκειμένου να εμπεδώσετε τη χρήση των συναρτήσεων $fftshift$ και $ifftshift$.


In [ ]:
import numpy as np

# Create the array X
X = np.arange(-2, 3)  # equivalent to MATLAB's -2:2

# Apply fftshift and ifftshift
fftshifted_X = np.fft.fftshift(X)
ifftshifted_X = np.fft.ifftshift(X)

# Double fftshift and a combination of fftshift and ifftshift
Y = np.fft.fftshift(np.fft.fftshift(X))
Z = np.fft.ifftshift(np.fft.fftshift(X))

# Check if arrays are equal
X_equals_Y = np.array_equal(X, Y)
X_equals_Z = np.array_equal(X, Z)

print(f"X equals Y: {X_equals_Y}")
print(f"X equals Z: {X_equals_Z}")

<details>
    <summary style="background-color: #3383FF; color: white; cursor: pointer; padding: 2px; border-radius: 5px;"><strong>Αντίστοιχος κώδικας MatLab</strong></summary>
    <div >
        <pre>
X = [-2:2]
fftshift(X)
ifftshift(X)
Y = fftshift(fftshift(X));
Z = ifftshift(fftshift(X));
isequal(X,Y)
isequal(X,Z)
        </pre>
    </div>
</details>



Ερώτηση 1: Ποιο εκ των διανυσμάτων Υ και Ζ ισούται με το X; Γράψτε την απάντησή σας σε ένα αρχείο κειμένου lab2_nnnnn.txt, όπου nnnnn τα πέντε τελευταία νούμερα του αριθμού μητρώου σας, χρησιμοποιώντας το Notepad από το μενού των Windows (Start → Programs → Accessories → Notepad) και αποθηκεύστε το στον φάκελο My Documents. Θα υποβάλετε το αρχείο αυτό ηλεκτρονικά στο τέλος, αφού απαντήσετε και τις επόμενες ερωτήσεις, οπότε μπορείτε να τα αφήσετε ανοικτό.
Ερώτηση 2: Επαναλάβατε με $X=[-1:2]$. Τι παρατηρείτε; Γράψτε την απάντησή σας στο αρχείο κειμένου lab2_nnnnn.txt.
Δοκιμάστε στο παράθυρο εντολών τα ακόλουθα δύο παραδείγματα για να εμπεδώσετε τη χρήση των συναρτήσεων $fftshift$ και $ifftshift$ σε συνδυασμό με τις $fft$ και $ifft$.




In [ ]:
import numpy as np
from scipy.fft import fft, ifft, fftshift, ifftshift
import dash
from dash import Dash, dcc, html
import plotly.graph_objs as go
import dash_bootstrap_components as dbc


# First part: Original signal and its FFT
xb = np.array([1, 2, 3, 4, 5, 4, 3, 2, 1])
x = ifftshift(xb)
X = fft(x)
Xb = fftshift(X)  # Spectrum with DC component in the center

# Second part: Low-pass filter effect
Xb_low_pass = np.array([0, 0, 1, 1, 1, 1, 1, 0, 0])
X_low_pass = ifftshift(Xb_low_pass)
x_low_pass = ifft(X_low_pass)
xb_low_pass = fftshift(x_low_pass)

app11 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Directly define the figures for plotting
Original_Signal = go.Figure(go.Scatter(x=np.arange(-4, 5), y=xb,line=dict(color='blue'), mode='lines+markers', name='Original Signal'))
Original_Signal.update_layout(
    title='Original Signal',
    xaxis={'title': 'Time (s)', 'showgrid': True, 'gridcolor': 'LightGrey'},
    yaxis={'title': 'Amplitude', 'showgrid': True, 'gridcolor': 'LightGrey'},
    margin={'l': 40, 'b': 40, 't': 40, 'r': 40},
    hovermode='closest',
    paper_bgcolor='white',
    plot_bgcolor='white',
    template='plotly_white'
)

Magnitude_Spectrum = go.Figure(go.Scatter(x=np.arange(-4, 5), y=np.abs(Xb),line=dict(color='red'), mode='lines+markers', name='Magnitude Spectrum'))
Magnitude_Spectrum.update_layout(
    title='Magnitude Spectrum',
    xaxis={'title': 'Frequency (Hz)', 'showgrid': True, 'gridcolor': 'LightGrey'},
    yaxis={'title': 'Amplitude', 'showgrid': True, 'gridcolor': 'LightGrey'},
    margin={'l': 40, 'b': 40, 't': 40, 'r': 40},
    hovermode='closest',
    paper_bgcolor='white',
    plot_bgcolor='white',
    template='plotly_white'
)

Low_pass_Spectrum = go.Figure(go.Scatter(x=np.arange(-4, 5), y=Xb_low_pass,line=dict(color='green'), mode='lines+markers', name='Low-pass Spectrum'))
Low_pass_Spectrum.update_layout(
    title='Low-pass Spectrum',
    xaxis={'title': 'Time (s)', 'showgrid': True, 'gridcolor': 'LightGrey'},
    yaxis={'title': 'Amplitude', 'showgrid': True, 'gridcolor': 'LightGrey'},
    margin={'l': 40, 'b': 40, 't': 40, 'r': 40},
    hovermode='closest',
    paper_bgcolor='white',
    plot_bgcolor='white',
    template='plotly_white'
)

Reconstructed_Signal = go.Figure(go.Scatter(x=np.arange(-4, 5), y=xb_low_pass.real,line=dict(color='purple'), mode='lines+markers', name='Reconstructed Signal'))
Reconstructed_Signal.update_layout(
    title='Reconstructed Signal',
    xaxis={'title': 'Frequency (Hz)', 'showgrid': True, 'gridcolor': 'LightGrey'},
    yaxis={'title': 'Amplitude', 'showgrid': True, 'gridcolor': 'LightGrey'},
    margin={'l': 40, 'b': 40, 't': 40, 'r': 40},
    hovermode='closest',
    paper_bgcolor='white',
    plot_bgcolor='white',
    template='plotly_white'
)

app11.layout = html.Div(
    children=[
        html.H2('Signal Analysis'),
        dbc.Row(
            children=[
                dbc.Col(dcc.Graph(figure=Original_Signal, style={'height': '250px'}), width=6),
                dbc.Col(dcc.Graph(figure=Magnitude_Spectrum, style={'height': '250px'}), width=6)
            ]
        ),
        dbc.Row(
            children=[
                dbc.Col(dcc.Graph(figure=Low_pass_Spectrum, style={'height': '250px'}), width=6),
                dbc.Col(dcc.Graph(figure=Reconstructed_Signal, style={'height': '250px'}), width=6)
            ]
        )
    ],
    style={'backgroundColor': 'white', 'padding': '20px'}
)

if __name__ == '__main__':
    app11.run_server(mode='inline',debug=True, port=8200)


<details>
    <summary style="background-color: #3383FF; color: white; cursor: pointer; padding: 2px; border-radius: 5px;"><strong>Αντίστοιχος κώδικας MatLab</strong></summary>
    <div>
        <pre>
close all;
clear all;
clc;
xb=[1 2 3 4 5 4 3 2 1]                          % πραγματικό σήμα με άρτια συμμετρία
figure;
subplot (2,1,1);
plot([-4:4],xb);
ylabel('xb');
x=ifftshift(xb)                                 % το σήμα με τις αρνητικές συνιστώσες στο άνω μέρος
X=fft(x)                                        % FFT
Xb=fftshift(X)                                  % το φάσμα με τη dc συνιστώσα στο κέντρο, πραγματικές
                                                % τιμές με άρτια συμμετρία όπως αναμένεται
subplot (2,1,2);
plot([-4:4],Xb);
ylabel('Xb');
close all;
clear all;
clc;
Xb=[0 0 1 1 1 1 1 0 0]                          % φάσμα βαθυπερατού σήματος με άρτια συμμετρία
figure;
subplot (2,1,1);
plot([-4:4],Xb);
ylabel('Xb');
X=ifftshift(Xb)                                 % το φάσμα με τις αρνητικές συνιστώσες στο άνω μέρος
x=ifft(X)                                       % IFFT
xb=fftshift(x)                                  % πραγματικό σήμα με άρτια συμμετρία όπως αναμένεται
subplot (2,1,2);
plot([-4:4],xb);
ylabel('xb');
        </pre>
    </div>
</details>



Ερώτηση 3: Τροποποιείστε το προηγούμενο παράδειγμα ώστε να ξεκινήσετε απευθείας με τον ορισμό του φάσματος του βαθυπερατού σήματος $X$ όπως το αναμένει η $ifft$. Γράψτε την απάντησή σας στο αρχείο κειμένου lab2_nnnnn.txt.

In [ ]:
from scipy import signal
import scipy.io.wavfile
# Ανατρέξτε στην τεκμηρίωση της βιβλιοθήκης scipy.signal
# https://docs.scipy.org/doc/scipy/reference/signal.html
from scipy.fft import fft, fftfreq
from dash import Dash, dcc, html, Input, Output
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import pandas
import plotly
import plotly.express as px
import plotly.graph_objs as go
import dash_bootstrap_components as dbc
import warnings
warnings.filterwarnings('ignore')
print("Libraries added successfully!")

## Μέρος 2: Σχεδιασμός και υλοποίηση φίλτρων

Θα ασχοληθούμε με το Παράδειγμα 1.2 της παραγράφου 1.5 του τεύχους Μαθήματος. Το παράδειγμα αυτό παρουσιάζει δύο εναλλακτικές μεθόδους σχεδιασμού FIR φίλτρων:
α) τη μέθοδο των παραθύρων και
β) τη μέθοδο των ισοϋψών κυματώσεων
τις οποίες εφαρμόζει στην περίπτωση βαθυπερατών φίλτρων. 

Στο παράδειγμα, τα φίλτρα δοκιμάζονται σε ένα πραγματικό σήμα, s, το οποίο είναι αποθηκευμένο  στο αρχείο sima.mat (binary αρχείο MATLAB). Πρόκειται για ένα σήμα sonar με φάσμα που εκτείνεται μέχρι περίπου τα 4 KHz και συχνότητα δειγματοληψίας Fs=8192 (είναι και αυτή αποθηκευμένη στο αρχείο sima.mat, μαζί με το σήμα).

Εδώ θα πειραματιστούμε με δύο σήματα: (i) το sonar του παραδείγματος, το οποίο εδώ διαβάζεται από ένα .txt αρχείο (έχει προέλθει με εξαγωγή του s από το MATLAB) και (ii) ένα σήμα μουσικής, το violin.wav (σήμα από μουσική βιολιού), το οποίο περιέχει υψηλότερες συχνότητες και έχει προέλθει με δειγματοληψία στα Fs_viol=44100 Hz.


### Σήμα sonar

In [ ]:
# Ανάγνωση δειγμάτων σήματος από txt file
with open('files/sima.txt') as f:
    s = [float(x) for x in f]
s=np.array(s)   
print('μέγεθος σήματος=', s.shape)
Fs=8192

#### Στο πεδίο του χρόνου

In [ ]:
import plotly.graph_objs as go
import numpy as np
from IPython.core.display import HTML
import plotly.io as pio

# Assuming s and Fs are defined earlier in your notebook
t = np.arange(0, len(s)) / Fs

# Time domain plot of x
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=s, mode='lines', line=dict(color='#00CC96')))
fig.update_layout(title='Time domain plot of x', title_x=0.5, title_font=dict(size=20, color='black', family="Arial, sans-serif"),
                  xaxis_title='t (sec)',
                  yaxis_title='Amplitude',
                  template='plotly_white',
                  xaxis=dict(range=[0, 0.8]),
                  yaxis=dict(range=[-0.05, 0.05]))
fig.show(renderer='notebook')

# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))

#### Ακούμε το σήμα 

In [ ]:
import ipywidgets as widgets
from IPython.display import Audio, display

# Assuming `s` is your audio signal and `Fs` is the sampling rate
# If `s` is not defined, you need to define it or load an audio signal.

# Function to play the sound
def play_sound(b):
    display(Audio(data=20*s, rate=Fs))

# Create a button widget
button1 = widgets.Button(description="Play Sound")

# Display the button
display(button1)

# On button click, play the sound
button1.on_click(play_sound)

#### Φάσμα (spectrum) 

In [ ]:
f, Pxx_den = signal.welch(s, Fs, noverlap=128, nperseg=256)

# Create Plotly figure
fig = go.Figure()

# Add trace for the PSD using a logarithmic scale on the y-axis
fig.add_trace(go.Scatter(x=f, y=Pxx_den, mode='lines',line=dict(color='#1F77B4')))

# Update layout for a semilog plot
fig.update_layout(
    title='Φάσμα σήματος sonar',title_x=0.5, title_font=dict(size=20, color='black', family="Arial, sans-serif"),
    xaxis_title='Συχνότητα (Hz)',
    yaxis_title='Πυκνότητα φάσματος ισχύος',
    yaxis_type='log',  # Set y-axis to logarithmic scale
    template='plotly_white',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),  # Customize grid color
)

# Show figure
fig.show(renderer='notebook')
# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))

### Σήμα βιολιού

In [ ]:
f=open("files/violin.wav", 'rb')
Fs_viol, s_viol = scipy.io.wavfile.read(f)
print('Fs_viol=',Fs_viol, ' number of samples=',len(s_viol))
f.close()

#### Στο πεδίο του χρόνου

In [ ]:
tvl = np.arange(0, len(s_viol)) / Fs_viol

# Create Plotly figure
fig = go.Figure()

# Add trace for the signal
fig.add_trace(go.Scatter(x=tvl, y=s_viol, mode='lines',line=dict(color='#1F77B4')))

# Update layout
fig.update_layout(
    title='Signal over time',title_x=0.5, title_font=dict(size=20, color='black', family="Arial, sans-serif"),
    xaxis_title='Time (s)',
    yaxis_title='Amplitude',
    template='plotly_white',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

# Show figure
fig.show(renderer='notebook')
# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import sounddevice as sd
import threading

# Define your sound and its sampling rate
# s_viol and Fs_viol should be defined before this point

def play_sound():
    sd.play(s_viol, Fs_viol)

def on_button_clicked(b):
    threading.Thread(target=play_sound).start()

# Create a button widget
play_button = widgets.Button(description="Play Sound")

# Link the button to the function that plays the sound
play_button.on_click(on_button_clicked)

# Display the button
display(play_button)


#### Φάσμα (spectrum) και Φασματόγραμμα  (spectorgram)

In [ ]:
f, Pxx_den = signal.welch(s_viol, Fs_viol, nperseg=1024, noverlap=256)

# Create Plotly figure
fig = go.Figure()

# Add trace for the PSD using a logarithmic scale on the y-axis
fig.add_trace(go.Scatter(x=f, y=Pxx_den, mode='lines',line=dict(color='#1F77B4')))

# Update layout for a semilog plot
fig.update_layout(
    title='Power Spectral Density', title_x=0.5, title_font=dict(size=20, color='black', family="Arial, sans-serif"),
    xaxis_title='Συχνότητα [Hz]',
    yaxis_title='Πυκνότητα φάσματος ισχύος [V<sup>2</sup>/Hz]',
    yaxis_type='log',  # Set y-axis to logarithmic scale
    template='plotly_white',
    xaxis=dict(),
    yaxis=dict(),  # Customize grid color
)

# Set y-axis range
fig.update_yaxes(range=[np.log10(0.5e-2), np.log10(1e5)])

# Show figure
fig.show(renderer='notebook')
# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))

In [ ]:
f, tsp, Sxx = signal.spectrogram(s, Fs)

# Create Plotly figure
fig = go.Figure(data=go.Heatmap(
    z=10 * np.log10(Sxx),  # Convert power to dB
    x=tsp,  # Time for x-axis
    y=f,  # Frequency for y-axis
))

# Update layout
fig.update_layout(
    title='Spectrogram', title_x=0.5, title_font=dict(size=20, color='black', family="Arial, sans-serif"),
    xaxis_title='Χρόνος [sec]',
    yaxis_title='Συχνότητα [Hz]',
    template='plotly_white'
)

# Show figure
fig.show(renderer='notebook')
# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))

### Βαθυπερατά φίλτρα

#### Η μέθοδος των παραθύρων
...
...


In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt


H = np.hstack((np.ones(int(Fs/8)), np.zeros(int(Fs-Fs/4)), np.ones(int(Fs/8))))

# Create the x values for the stem plot
x_values = np.arange(len(H))

# Create Plotly figure
fig = go.Figure()

# Add stems
for x, y in zip(x_values, H):
    fig.add_trace(go.Scatter(x=[x, x], y=[0, y], mode='lines', line=dict(color='#1F77B4')))

# Add markers
fig.add_trace(go.Scatter(x=x_values, y=H, mode='markers', marker=dict(color='#1F77B4')))

# Update layout
fig.update_layout(
    title='Stem plot', title_x=0.5,title_font=dict(size=20, color='black', family="Arial, sans-serif"),
    xaxis_title='Index',
    yaxis_title='Amplitude',
    template='plotly_white'
)

# Show figure
fig.show(renderer='notebook')
# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))


#its kinda slow


#### _Ορθογωνικό παράθυρο (απλή περικοπή της h)_

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np
import plotly.graph_objs as go

h = np.real(np.fft.ifft(H))
middle = int(len(h)/2)
h = np.hstack((h[middle:], h[:middle]))
h32 = h[middle-16:middle+16]
h64 = h[middle-32:middle+32]
h128 = h[middle-64:middle+64]
h160 = h[middle-80:middle+80]
h256 = h[middle-128:middle+128]


h_variants = {
    'h32': h[middle-16:middle+16],
    'h64': h[middle-32:middle+32],
    'h128': h[middle-64:middle+64],
    'h160': h[middle-80:middle+80],
    'h256': h[middle-128:middle+128],
}

# Function to create and update the stem plot based on selected filter
def create_update_stem_plot(h_data):
    x_values = np.arange(len(h_data))
    fig = go.Figure()
    
    # Add stems
    for x, y in zip(x_values, h_data):
        fig.add_trace(go.Scatter(x=[x, x], y=[0, y], mode='lines', line=dict(color='#1F77B4')))
    
    # Add markers
    fig.add_trace(go.Scatter(x=x_values, y=h_data, mode='markers', marker=dict(color='#1F77B4')))
    
    # Update layout
    fig.update_layout(
        title='Stem plot of selected filter',
        xaxis_title='Index',
        yaxis_title='Amplitude',
        template='plotly_white'
    )
    return fig

# Initialize the Dash app
app21 = dash.Dash(__name__)

# App layout
app21.layout = html.Div([
    html.Label('Filter:'),  # Label for the dropdown
    dcc.Dropdown(
        id='filter-dropdown',
        options=[{'label': key, 'value': key} for key in h_variants.keys()],
        value='h32',  # Default value
    ),
    dcc.Graph(id='plot')
])

# Callback to update the plot based on the dropdown selection
@app21.callback(
    Output('plot', 'figure'),
    [Input('filter-dropdown', 'value')]
)
def update_plot(selected_filter):
    h_data = h_variants[selected_filter]
    return create_update_stem_plot(h_data)

# Run the app
if __name__ == '__main__':
    app21.run_server(mode='inline',debug=True, port=8071)  # You can change the port number if needed


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objs as go
from scipy import signal
import numpy as np

# Compute frequency responses
freq32, resp32 = signal.freqz(h32)
freq64, resp64 = signal.freqz(h64)
freq128, resp128 = signal.freqz(h128)
freq160, resp160 = signal.freqz(h160)
freq256, resp256 = signal.freqz(h256)

# Compute frequency responses
freqs = {}
resps = {}
for filt in [32, 64, 128, 160, 256]:
    freqs[f'h{filt}'], resps[f'h{filt}'] = signal.freqz(eval(f'h{filt}'))

app22 = Dash(__name__)

app22.layout = html.Div([
    # Title for the checklist
     html.H4('Select Filter Size:', style={
        'textAlign': 'center',
        'marginBottom': '10px',
        'padding': '10px',  # Add some padding around the text
        'borderRadius': '5px',  # Optional: round the corners of the background
        'fontSize': '24px'  # Increase font size
    }),
    
    # Checklist with custom styling for bigger checkboxes
    dcc.Checklist(
        id='filter-size-checklist',
        options=[{'label': f'h{filt}', 'value': f'h{filt}'} for filt in [32, 64, 128, 160, 256]],
        value=['h32'],  # Default selected value
        labelStyle={'display': 'inline-block', 'marginRight': '20px', 'transform': 'scale(1.5)', 'marginBottom': '10px'},  # Customize checkbox size
        inputStyle={"marginRight": "5px"}  # Adjust space between checkbox and label
    ),
    dcc.Graph(id='frequency-response-plot')
])

# Callback to update the frequency response plot
@app22.callback(
    Output('frequency-response-plot', 'figure'),
    [Input('filter-size-checklist', 'value')]
)
def update_frequency_response(selected_filters):
    fig = go.Figure()
    
    for filt in selected_filters:
        fig.add_trace(go.Scatter(
            x=0.5 * Fs * freqs[filt] / np.pi,
            y=np.abs(resps[filt]),
            mode='lines',
            name=filt
        ))
    
    fig.update_layout(
        title='Frequency Response of Selected Filters',
        xaxis_title='Frequency (Hz)',
        yaxis_title='Gain',
        yaxis_type='log',
        template='plotly_white'
    )
    
    return fig

if __name__ == '__main__':
    app22.run_server(mode='inline',debug=True, port=8072)

#### _Παράθυρα Hamming και Kaiser_

In [ ]:
w_hamming=signal.hamming(len(h64))
h64_hamming = np.multiply(h64,w_hamming)
w_kaiser=signal.kaiser(len(h64),5)
h64_kaiser = np.multiply(h64,w_kaiser)

freq,resp64_hamming = signal.freqz(h64_hamming)
freq,resp64_kaiser = signal.freqz(h64_kaiser)

fig = go.Figure()

# Add trace for h64 (rectangular window)
fig.add_trace(go.Scatter(x=0.5 * Fs * freq / np.pi, y=np.abs(resp64), mode='lines', name='Rectangular', line=dict(color='#1F77B4')))

# Add trace for h64 with Hamming window
fig.add_trace(go.Scatter(x=0.5 * Fs * freq / np.pi, y=np.abs(resp64_hamming), mode='lines', name='Hamming', line=dict(color='green')))

# Add trace for h64 with Kaiser window
fig.add_trace(go.Scatter(x=0.5 * Fs * freq / np.pi, y=np.abs(resp64_kaiser), mode='lines', name='Kaiser', line=dict(color='red')))

# Update layout for a semilog plot
fig.update_layout(
    title='Απόκριση συχνότητας βαθυπερατού φίλτρου<br>παράθυρα hamming (πράσινο) και kaiser (κόκκινο)<br>(το αντίστοιχο ορθογωνικό σε μπλε)', title_x=0.5,title_font=dict(size=20, color='black', family="Arial, sans-serif"),
    xaxis_title='Συχνότητα (Hz)',
    yaxis_title='Κέρδος',
    yaxis_type='log',  # Set y-axis to logarithmic scale
    template='plotly_white',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

# Show figure
fig.show(renderer='notebook')
# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))

#### Φίλτρα ισοϋψών κυματώσεων

In [ ]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import numpy as np
from scipy import signal


# Define the filters using the Remez algorithm
filters = {
    'Equiripple Filter 32+1': signal.remez(32, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs),
    'Equiripple Filter 64+1': signal.remez(64, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs),
    'Equiripple Filter 128+1': signal.remez(128, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs),
    'Equiripple Filter 160+1': signal.remez(160, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs),
    'Equiripple Filter 256+1': signal.remez(256, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs),
}

app23 = Dash(__name__)

app23.layout = html.Div([
    # Title
    html.H4('Select Equiripple Filter Length:', style={
        'textAlign': 'center',
        'marginBottom': '10px',
        'padding': '10px',  # Add some padding around the text
        'borderRadius': '5px',  # Optional: round the corners of the background
        'fontSize': '24px'  # Increase font size
    }),
    
    # Checklist for selecting filters
    dcc.Checklist(
        id='filter-checklist',
        options=[{'label': name, 'value': name} for name in filters.keys()],
        value=[],  # No default selection
        labelStyle={'display': 'block'},  # Display checkboxes in block style for better readability
        inputStyle={'display': 'block',"marginRight": "10px", "transform": "scale(1.5)"},  # Make checkboxes slightly larger
    ),
    
    # Graph for displaying frequency response
    dcc.Graph(id='frequency-response-plot'),
])

@app23.callback(
    Output('frequency-response-plot', 'figure'),
    [Input('filter-checklist', 'value')]
)
def update_frequency_response(selected_filters):
    fig = go.Figure()
    
    for filter_name in selected_filters:
        filter_coeffs = filters[filter_name]
        freq, response = signal.freqz(filter_coeffs, fs=Fs)
        fig.add_trace(go.Scatter(x=freq, y=20 * np.log10(np.abs(response)), mode='lines', name=filter_name))
    
    fig.update_layout(
        title='Frequency Response of Selected Equiripple Filters',
        xaxis_title='Frequency (Hz)',
        yaxis_title='Gain (dB)',
        template='plotly_white'
    )
    return fig

if __name__ == '__main__':
    app23.run_server(mode='inline',debug=True, port=8073)


#### Εφαρμογή του φίλτρου

In [ ]:

from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import numpy as np
from scipy import signal
from scipy.io.wavfile import write


# Design the filters (replace with your actual filter design)
lp_filters = {
    'lpass32': signal.remez(32, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs),
    'lpass64': signal.remez(64, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs),
    'lpass128': signal.remez(128, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs),
    'lpass160': signal.remez(160, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs),
    'lpass256': signal.remez(256, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs)
} 

for name, lp_filter_coeffs in lp_filters.items():
    # Apply the filter to the signal
    s_lp_filtered = signal.convolve(s, lp_filter_coeffs, mode='same') / np.sum(lp_filter_coeffs)

    # Generate a filename based on the filter name
    filename = f'assets/{name}.wav'

    # Normalize the filtered signal to prevent clipping
    s_lp_filtered_normalized = np.int16((s_lp_filtered / np.max(np.abs(s_lp_filtered))) * 32767)

    # Save the filtered signal as a WAV file
    write(filename, Fs, s_lp_filtered_normalized)

app24 = Dash(__name__)

app24.layout = html.Div([
    dcc.Dropdown(
        id='filter-dropdown',
        options=[{'label': name, 'value': name} for name in lp_filters.keys()],
        value='lpass64'
    ),
    dcc.Graph(id='psd-plot'),
    html.Audio(id='audio-player', controls=True, src='assets/lpass32.wav')
])

@app24.callback(
    Output('psd-plot', 'figure'),
    [Input('filter-dropdown', 'value')]
)
def update_psd_plot(selected_lp_filter_name):
    selected_lp_filter = lp_filters[selected_lp_filter_name]
    s_lp_filtered = signal.convolve(s, selected_lp_filter, mode='same') / np.sum(selected_lp_filter)
    f, Pxx_den_lp = signal.welch(s_lp_filtered, Fs, noverlap=128, nperseg=256)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=f, y=Pxx_den_lp, mode='lines', line=dict(color='#1F77B4')))
    fig.update_layout(
        title=f'Φάσμα φιλτραρισμένου σήματος: {selected_lp_filter_name}',
        xaxis_title='Συχνότητα (Hz)',
        yaxis_title='Πυκνότητα φάσματος ισχύος',
        yaxis_type='log',
        template='plotly_white'
    )
    return fig

@app24.callback(
    Output('audio-player', 'src'),
    [Input('filter-dropdown', 'value')]
)
def update_audio_source(selected_lp_filter_name):
    return f'/assets/{selected_lp_filter_name}.wav'

if __name__ == '__main__':
    app24.run_server(mode='inline',debug=True, port=8074)

In [ ]:
# Παράμετροι σήματος
Fs = 8192  # Συχνότητα δειγματοληψίας
t = np.arange(0, 1.0, 1/Fs)  # Διάρκεια σήματος 1 δευτερόλεπτο

# Δημιουργία του σήματος
s1 = np.sin(2*np.pi*700*t) + np.sin(2*np.pi*900*t) + np.sin(2*np.pi*1400*t) + np.sin(2*np.pi*2500*t)

# Σχεδιασμός των φίλτρων Parks-McClellan
lpass32 = signal.remez(32, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs)
lpass64 = signal.remez(64, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs)
lpass128 = signal.remez(128, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs)
lpass160 = signal.remez(160, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs)
lpass256 = signal.remez(256, [0, 1000, 1300, Fs/2], [1, 0], fs=Fs)

# Φιλτράρισμα του σήματος
s1_filtered32 = signal.lfilter(lpass32, 1.0, s1)
s1_filtered64 = signal.lfilter(lpass64, 1.0, s1)
s1_filtered128 = signal.lfilter(lpass128, 1.0, s1)
s1_filtered160 = signal.lfilter(lpass160, 1.0, s1)
s1_filtered256 = signal.lfilter(lpass256, 1.0, s1)

# Φασματική πυκνότητα των φιλτραρισμένων σημάτων
f, Pxx1_den = signal.welch(s1, Fs, nperseg=1024)
f32, Pxx1_den32 = signal.welch(s1_filtered32, Fs, nperseg=1024)
f64, Pxx1_den64 = signal.welch(s1_filtered64, Fs, nperseg=1024)
f128, Pxx1_den128 = signal.welch(s1_filtered128, Fs, nperseg=1024)
f160, Pxx1_den160 = signal.welch(s1_filtered160, Fs, nperseg=1024)
f256, Pxx1_den256 = signal.welch(s1_filtered256, Fs, nperseg=1024)

# Σχεδίαση των φασματικών πυκνοτήτων σε Plotly
fig = go.Figure()


# Αρχικό σήμα
fig.add_trace(go.Scatter(x=f, y=10*np.log10(Pxx1_den), mode='lines', name='Original Signal'))

# Φιλτραρισμένο σήμα 32
fig.add_trace(go.Scatter(x=f32, y=10*np.log10(Pxx1_den32), mode='lines', name='Filtered Signal 32'))

# Φιλτραρισμένο σήμα 64
fig.add_trace(go.Scatter(x=f64, y=10*np.log10(Pxx1_den64), mode='lines', name='Filtered Signal 64'))

# Φιλτραρισμένο σήμα 128
fig.add_trace(go.Scatter(x=f128, y=10*np.log10(Pxx1_den128), mode='lines', name='Filtered Signal 128'))

# Φιλτραρισμένο σήμα 160
fig.add_trace(go.Scatter(x=f160, y=10*np.log10(Pxx1_den160), mode='lines', name='Filtered Signal 256'))

# Ενημέρωση layout
fig.update_layout(title='Φασματική Πυκνότητα Ισχύος',
                  xaxis_title='Συχνότητα (Hz)',
                  yaxis_title='Πυκνότητα Ισχύος (dB)',
                  template='plotly_white')

fig.show(renderer='notebook')
# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))


### Ζωνοπερατά φίλτρα

#### _Με αναλυτικό υπολογισμό της κρουστικής απόκρισης και παράθυρο_ 

In [ ]:
# Με αναλυτικό υπολογισμό της κρουστικής απόκρισης και παράθυρο kaiser
f1=700; f2=1500;  
Ts=1/Fs
f2m1=(f2-f1); f2p1=(f2+f1)/2; N=256
t=np.arange(-(N-1),N-1,2)*Ts/2
hbp=2/Fs*np.divide(np.multiply(np.cos(2*np.pi*f2p1*t),np.sin(np.pi*f2m1*t))/np.pi,t)
hbpw=np.multiply(hbp,signal.kaiser(len(hbp),5))

s1_bp=signal.convolve(s1,hbp,'same')

In [ ]:
f0, Pxx1_den = signal.welch(s1_bp, Fs, noverlap=128, nperseg=256)

# Create Plotly figure
fig = go.Figure()

# Add trace for the power spectral density
fig.add_trace(go.Scatter(x=f0, y=Pxx1_den, mode='lines', line=dict(color='blue')))

# Update layout for a semilog plot
fig.update_layout(
    title='Φάσμα ζωνοπερατού σήματος sonar',title_x=0.5,title_font=dict(size=20, color='black', family="Arial, sans-serif"),
    xaxis_title='Συχνότητα (Hz)',
    yaxis_title='Πυκνότητα φάσματος ισχύος',
    yaxis_type='log',  # Set y-axis to logarithmic scale
    template='plotly_white',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True, range=[np.log10(1e-15), np.log10(1e-7)])  # Adjust y-axis range
)

# Show figure
fig.show(renderer='notebook')
# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))

def play_sound(b):
    sd.play(20 * s1_bp, Fs)

# Create a button widget
play_button = widgets.Button(description="Play Sound")

# Link the button to the function that plays the sound
play_button.on_click(play_sound)

# Display the button
display(play_button)

#### _Ζωνοπερατό ισουψών κυματώσεων_ 

In [ ]:
bpass = signal.remez(128, [0, f1*0.95, f1*1.1, f2*0.95, f2*1.05, Fs/2], [0, 1, 0], fs=Fs)
freq,resp_pm = signal.freqz(bpass)

# Compute frequency response for the equiripple bandpass filter
freq, resp_pm = signal.freqz(bpass)

# Create Plotly figure
fig = go.Figure()

# Add trace for equiripple bandpass filter response
fig.add_trace(go.Scatter(x=0.5 * Fs * freq / np.pi, y=np.abs(resp_pm), mode='lines', name='Equiripple Bandpass', line=dict(color='green')))

# Update layout for a semilog plot
fig.update_layout(
    title='Απόκριση συχνότητας ζωνοπερατού φίλτρου equirriple',title_x=0.5,title_font=dict(size=20, color='black', family="Arial, sans-serif"),
    xaxis_title='Συχνότητα (Hz)',
    yaxis_title='Κέρδος',
    yaxis_type='log',  # Set y-axis to logarithmic scale
    template='plotly_white',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

# Show figure
fig.show(renderer='notebook')

# Convert figure to HTML
fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
display(HTML(fig_html))